<h1 align="center"> TP6 : Implementation of variational damage gradient models for brittle fracture </h1>
<h2 align="center"> Jeremy Bleyer, Corrado Maurini </h2>

> **Objectives**:
> * Implement variational damage gradient models for brittle fracture
> * Check the numerical behaviour of localized 1D solutions 
> * Study the influence of the regularization length scale and the mesh size on the solution

We consider the gradient damage models presented in class: 
\begin{equation}
\mathcal{E}(u,d) = \mathcal{E}_{el}(u,d) + \mathcal{E}_f(u,d) = \int_\Omega \psi(\boldsymbol{\varepsilon}(u), d) \,\text{d}\Omega + \int_\Omega \dfrac{G_c}{\ell_0 c_w}\left(w(d)+\ell_0^2\|\nabla d\|^2\right)\,\text{d}\Omega
\end{equation}
with an elastic energy density of the form:
\begin{equation}
\psi(\boldsymbol{\varepsilon},d) = \left((1-d)^2+\kappa_{res}\right)\dfrac{1}{2}\boldsymbol{\varepsilon}:\mathbb{C}_0:\boldsymbol{\varepsilon}
\end{equation}
and the two modelsfor the fracture energy:
\begin{equation}
\textsf{AT1}\qquad w(d)=d \quad; \quad c_w = \dfrac{8}{3} \tag{AT1}
\end{equation}
or
\begin{equation}
\textsf{AT2}\qquad w(d)=d^2 \quad; \quad c_w = 2 \tag{AT2}
\end{equation}

## Implementation

All the simulations are done in plane stress conditions

1. In `damage_gradient.py`, define using UFL operators the following functions:
* `sigma(v, d)` computing the damaged elastic stress as a function of `u`. You can use the already-define `eps(v)` function for the strain and `sigma0(v)` for the non-damaged stress.
* `g(d)` computing the degradation function as a function of a damage field `d`. You can use the already-defined residual stiffness constant `kres`.
* `psi(v, d)` computing the damaged elastic strain energy density as a function of `u` and `d`
* the forms `elastic_energy` and `fracture_energy` corresponding to $\mathcal{E}_\text{el}$ and $\mathcal{E}_\text{frac}$ as a function of the unknown displacement `u` and damage field `d`

2. Run the simulation to check the behaviour of the homogeneous solution for both AT1 and AT2 models.


## Localized 1D solutions

We consider the same problem with $\ell_0 = 0.1$ and now `refinement_level=4`.
A refinement of level `k` yields a mesh with an element size of roughly $2^{-k}L$.
1. Launch the computation for AT1 and `Umax=3e-3`. What happens ?
2. Rerun the computation by imposing now `damage_bcs=True`. This keyword enforces the damage field to be zero at both extremities where displacement is fixed. What happens ?
3. What is the value of the dissipated energy ? What value do you expect theoretically for a localized solution ? Report the value of the dissipated energy for `refinement_level=5,6,7` and comment.
4. Do the same with $\ell_0 = 0.02$ and `Umax=5e-3`. Compare with the previous case.
5. Now, what happens for the AT2 model ?


## Notched tension
We now consider `refinement_level=0`, `Umax=2.5e-3` and `damage_bcs=False`. We also change the problem type to `problem="notched"` corresponding to a similar plate in tension (same boundary conditions), of dimensions $L × H = 1 × 0.5$, perforated by two notches of radius $R=$`notch_radius`, horizontal spacing (`notch_spacing`) and aspect ratio (`aspect_ratio`).

1. We take $R = 0.2$ and $\ell_0 = 0.1$. Run the computation for AT2 and comment. What happens upon mesh refinement?
2. What can we say for $\ell_0 = 0.05, 0.025$ ?
3. Now change to $R = 0.2$ and `aspect_ratio=100`. What is the influence of $\ell_0$ in this case compared to the previous case ?


## Cracking in mode II
We now consider `problem="shear"` with $L=H=1$.
This problem corresponds to a shear loading of a cut-out block by applying an horizontal
displacement on the top face, the bottom face being fixed. The pre-crack is therefore loaded
initially in mode II.

1. First change the boundary conditions accordingly: `facets=2` is the bottom boundary, `facets=4` is the top boundary. Run the computation for the AT1 model with $\ell_0 = 0.04$, `Nincr = 50` and `Umax=6e-3`. What can we observe regarding the crack path ? 
2. By considering how the top block
moves, what can you say about the top crack ? What should happen in reality ?
3. Amor et al. proposed the following tension/compression splitting model:
\begin{align}
\psi(\boldsymbol{\varepsilon},d) &= g(d)\psi_+(\boldsymbol{\varepsilon}) + \psi_-(\boldsymbol{\varepsilon})\\
\psi_+(\boldsymbol{\varepsilon}) &= \dfrac{1}{2}\kappa\langle\textrm{tr}(\boldsymbol{\varepsilon})\rangle_+^2 + \mu\textrm{dev}(\boldsymbol{\varepsilon}):\textrm{dev}(\boldsymbol{\varepsilon})\\
\psi_-(\boldsymbol{\varepsilon}) &= \dfrac{1}{2}\kappa\langle\textrm{tr}(\boldsymbol{\varepsilon})\rangle_-^2 
\end{align}
Adapt the value of the elastic energy using the functions `psi_pos` and `psi_neg` and run again the simulation with this model.

4. What happens when using the AT2 model ? What can we expect upon mesh refinement and smaller regularization length ?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from damage_gradient import solve_problem
from mealor.utils import plot_over_line
from mealor.plotting import plot_mesh
from generate_mesh import setup_geometry

L = 1.0
H = 0.1
problem_type = "bar"
domain, facets = setup_geometry(problem_type, L, H, refinement_level=0)
plot_mesh(domain)

In [ ]:
# Material parameters
E = 3.0e3
nu = 0.3
Gc = 3e-3
l0 = 0.1
model = "AT1"
Nincr = 50
Umax = 5e-3
mech_params = (E, nu, Gc, l0, model, Umax, Nincr)

# boundary conditions
damage_bcs = True
def generate_bcs(dirichlet, Uimp):
    # ux = Uimp on right boundary
    dirichlet.add_bc_topological(facets, 3, ux=Uimp)
    # ux = 0 on left boundary
    dirichlet.add_bc_topological(facets, 1, ux=0)    
    # uy = 0 on x=y=0 point
    dirichlet.add_bc_geometrical(
        lambda x: np.logical_and(np.isclose(x[0], 0), np.isclose(x[1], 0)), uy=0
        )
    
# Alternate minimization parameters
Niter_max = 100 
tol = 1e-6

prob_params = (problem_type, generate_bcs, damage_bcs, Niter_max, tol)
results, u, d, sig = solve_problem(
    domain, facets, prob_params, mech_params
)

In [ ]:
Uimp = results[:, 0]
Force = results[:, 1]
elastic = results[:, 2]
fracture = results[:, 3]

S0 = 0.1
print("Final dissipated energy [/Gc)]", fracture[-1]/Gc)
print(f"Maximum stress {max(Force/S0)}")

### Plot stress curve and energies

In [ ]:
plt.plot(Uimp/L, Force/S0, '-k')
plt.xlabel("Average strain")
plt.ylabel("Average stress")
plt.show()

plt.figure()
plt.plot(Uimp/L, elastic, label="elastic")
plt.plot(Uimp/L, fracture, label="fracture")
plt.plot(Uimp/L, elastic + fracture, label="total")
plt.xlabel("Average strain")
plt.ylabel("Energies")
plt.legend()
plt.show()

### Plot localized solution profiles

In [ ]:
line = [(0., H / 2), (1.0, H / 2)]
dv, points = plot_over_line(d, line, domain)
x = points[:, 0]

pos = lambda x: (x+np.abs(x))/2.0

profile = {"AT2": np.exp(-np.abs(x - L / 2) / l0),
            "AT1": (pos(1-np.abs(x-L/2)/2/l0))**2}

plt.plot(x, dv, "-", label="damage profile")
plt.xlabel(r"$x$")
plt.ylabel(r"Damage $d$")
plt.ylim(0, 1)
plt.plot(x, profile[model], "--r", label=f"{model} theoretical profile")
plt.legend()
plt.show()